In [ ]:
import pickle as pkl
import matplotlib.pyplot as plt
import tqdm, re, pandas as pd, random, pprint

In [ ]:
def loadTxt(name):
    return [i.strip() for i in open(f'../data/{name}', 'r').readlines()]

In [ ]:
def someDump(df, name):
    temp= {
        'user': [], 'hashtagsCount': [], 'uniqueHashtagsCount': [], 'mentions': [], 'uniqueMentions': [], 'retweets': [],
        'averageHashtagsCount': [], 'averageUniqueHashtagsCount': [], 'averageMentions': [], 'averageUniqueMentions': [],
        'tweets': [], 'class': []
    }
    classUser= {
        'suspended': 0,
        'normal': 1,
        'restored': 2
    }
    
    for i in df:
        temp['user'].append(i)
        temp['tweets'].append(df[i]['tweets_count'])
        temp['retweets'].append(df[i]['retweets_count'])
        temp['class'].append(classUser[name])

        temp['hashtagsCount'].append(len(df[i]['hashtags']))
        temp['averageHashtagsCount'].append(len(df[i]['hashtags'])/df[i]['tweets_count'])
        temp['uniqueHashtagsCount'].append(len(set(df[i]['hashtags'])))
        temp['averageUniqueHashtagsCount'].append(len(set(df[i]['hashtags']))/df[i]['tweets_count'])

        temp['mentions'].append(len(df[i]['mentions']))
        temp['averageMentions'].append(len(df[i]['mentions'])/df[i]['tweets_count'])
        temp['uniqueMentions'].append(len(set(df[i]['mentions'])))
        temp['averageUniqueMentions'].append(len(set(df[i]['mentions']))/df[i]['tweets_count'])
        
    pd.DataFrame.from_dict(temp).to_csv(f'../data/network-{name}-features.csv', index= False)

In [ ]:
restoredUsersList= set()
for i in loadTxt('unsuspended_8_Oct.txt'):
    restoredUsersList.add(i)

In [ ]:
'''
   CSV's Column: UserID, Class, No. User Mentions, No. Unique User Mentions, No. Retweets, No. Hashtags, No. Unique Hashtags 
'''
normal= {}
with open(f'../data/all_tweets_normal_jan3_2021.pickle', 'rb') as f:
    try:
        while True:
            temp= pkl.load(f)
            for i in tqdm.tqdm(temp):
                user= i['user']['id_str']
                if i.get('retweeted_status'):
                    if normal.get(user) is None:
                        normal[user]= {
                            'hashtags': [j['text'] for j in i['entities']['hashtags']],
                            'mentions': [j['id_str'] for j in i['entities']['user_mentions']],
                            'tweets_count': 1,
                            'retweets_count': 1
                        }
                    else:
                        normal[user]['tweets_count']+= 1
                        normal[user]['retweets_count']+= 1
                elif i.get('full_text') or i.get('text'):
                    if normal.get(user) is None:
                        normal[user]= {
                            'hashtags': [j['text'] for j in i['entities']['hashtags']],
                            'mentions': [j['id_str'] for j in i['entities']['user_mentions']],
                            'tweets_count': 1,
                            'retweets_count': 0
                        }
                    else:
                        normal[user]['tweets_count']+= 1
                        for j in i['entities']['hashtags']:
                            normal[user]['hashtags'].append(j['text'])
                        for j in i['entities']['user_mentions']:
                            normal[user]['mentions'].append(j['id_str'])
                else:
                    print('--'*20)
                    pprint.pprint(i)
                    break
    except EOFError:
        pass

In [ ]:
'''
   CSV's Column: UserID, Class, No. User Mentions, No. Unique User Mentions, No. Retweets, No. Hashtags, No. Unique Hashtags 
'''
suspended= {}
restored= {}
with open(f'../data/all_tweets_suspended_jan3_2021.pickle', 'rb') as f:
    try:
        while True:
            temp= pkl.load(f)
            for i in tqdm.tqdm(temp):
                user= i['user']['id_str']
                if i.get('retweeted_status'):
                    if user in restoredUsersList:
                        if restored.get(user) is None:
                            restored[user]= {
                                'hashtags': [j['text'] for j in i['entities']['hashtags']],
                                'mentions': [j['id_str'] for j in i['entities']['user_mentions']],
                                'tweets_count': 1,
                                'retweets_count': 1
                            }
                        else:
                            restored[user]['tweets_count']+= 1
                            restored[user]['retweets_count']+= 1
                    else:
                        if suspended.get(user) is None:
                            suspended[user]= {
                                'hashtags': [j['text'] for j in i['entities']['hashtags']],
                                'mentions': [j['id_str'] for j in i['entities']['user_mentions']],
                                'tweets_count': 1,
                                'retweets_count': 1
                            }
                        else:
                            suspended[user]['tweets_count']+= 1
                            suspended[user]['retweets_count']+= 1
                elif i.get('full_text') or i.get('text'):
                    if user in restoredUsersList:
                        if restored.get(user) is None:
                            restored[user]= {
                                'hashtags': [j['text'] for j in i['entities']['hashtags']],
                                'mentions': [j['id_str'] for j in i['entities']['user_mentions']],
                                'tweets_count': 1,
                                'retweets_count': 0
                            }
                        else:
                            restored[user]['tweets_count']+= 1
                            for j in i['entities']['hashtags']:
                                restored[user]['hashtags'].append(j['text'])
                            for j in i['entities']['user_mentions']:
                                restored[user]['mentions'].append(j['id_str'])
                    else:
                        if suspended.get(user) is None:
                            suspended[user]= {
                                'hashtags': [j['text'] for j in i['entities']['hashtags']],
                                'mentions': [j['id_str'] for j in i['entities']['user_mentions']],
                                'tweets_count': 1,
                                'retweets_count': 0
                            }
                        else:
                            suspended[user]['tweets_count']+= 1
                            for j in i['entities']['hashtags']:
                                suspended[user]['hashtags'].append(j['text'])
                            for j in i['entities']['user_mentions']:
                                suspended[user]['mentions'].append(j['id_str'])
                else:
                    print('--'*20)
                    pprint.pprint(i)
                    break
    except EOFError:
        pass

In [ ]:
someDump(normal, 'normal')
someDump(suspended, 'suspended')
someDump(restored, 'restored')